In [ ]:
import joblib
import numpy as np
from http.server import BaseHTTPRequestHandler, HTTPServer

# Load the trained machine learning model
model = joblib.load('finalized_model.sav')

class MyHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            
            with open('index.html', 'rb') as file:
                self.wfile.write(file.read())
    
    def do_POST(self):
        if self.path == '/':
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length).decode('utf-8')
            
            # Extract and preprocess feature data from form submission
            gender = post_data.split('&')[0].split('=')[1]
            ssc_p = float(post_data.split('&')[1].split('=')[1])
            hsc_p = float(post_data.split('&')[2].split('=')[1])
            degree_p = float(post_data.split('&')[3].split('=')[1])
            workex = post_data.split('&')[4].split('=')[1]
            etest_p = float(post_data.split('&')[5].split('=')[1])
            specialisation = post_data.split('&')[6].split('=')[1]
            mba_p = float(post_data.split('&')[7].split('=')[1])
            hsc_s = post_data.split('&')[8].split('=')[1]
            degree_s = post_data.split('&')[9].split('=')[1]
           
            
            # Convert categorical features to binary values
            gender = 1 if gender == 'Male' else 0
            workex = 1 if workex == 'Yes' else 0
            specialisation = 1 if specialisation == 'Mkt&HR' else 0
            Arts = 0
            Science = 0
            Commerce = 0
            Sci_Tech = 0
            Comm_Mgmt = 0
            Others = 0
            if hsc_s == 'Arts':
                Arts = 1
                Science = 0
                Commerce = 0
            elif hsc_s == 'Science':
                Arts = 0
                Science = 1
                Commerce = 0
            elif hsc_s == 'Commerce':
                Arts = 0
                Science = 0
                Commerce = 1
            
            if degree_s == 'Science & Tech':
                Sci_Tech = 1
                Comm_Mgmt = 0
                Others = 0
            elif degree_s == 'Comm & Mngmt':
                Sci_Tech = 0
                Comm_Mgmt = 1
                Others = 0
            # Create input data for prediction
            input_data = np.array([[gender, ssc_p, hsc_p, degree_p, workex, etest_p, specialisation,
                                    mba_p, Arts, Commerce, Science, Comm_Mgmt, Others, Sci_Tech]])
            
            # Make a prediction using the model
            prediction = model.predict(input_data)
            
            # Format the prediction result
            result = "Placed" if prediction == 1 else "Not Placed"
            
            # Return the HTML response
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            response = f"<p>Prediction Result: {result}</p>".encode('utf-8')
            self.wfile.write(response)

# Create a simple HTTP server
with HTTPServer(("localhost", 8000), MyHandler) as httpd:
    print("Server started on port 8000")
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        httpd.server_close()
        print("Server stopped")


Server started on port 8000


127.0.0.1 - - [06/Aug/2023 18:46:35] "GET / HTTP/1.1" 200 -
